# Training own eng_pol mT5 translator 

input data was prepared in another notbook https://github.com/len-sla/NLP/blob/master/preparing_input_for_mT5.ipynb

---

Notbeook is based on the following sources:
https://towardsdatascience.com/how-to-train-an-mt5-model-for-translation-with-simple-transformers-30ba5fa66c5f
https://simpletransformers.ai/docs/tips-and-tricks/#saving-checkpoints

## 1. Mounting google drive and loading previously prepared input csv files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd

In [ ]:
train_df= pd.read_csv("/content/drive/MyDrive/mt5/data/eng-pol/train.tsv", sep="\t")
eval_df = pd.read_csv("/content/drive/MyDrive/mt5/data/eng-pol/eval.tsv", sep="\t")

In [ ]:
def change_df_col_toString(df):
  for i in df.columns:
    df[i] = df[i].astype('str')
  return df

In [ ]:
train_df=change_df_col_toString(train_df)
eval_df=change_df_col_toString(eval_df)

In [ ]:
train_df.sample(10)

,Unnamed: 0,prefix,input_text,target_text
1165946,1165946,translate polish to english,Nic.,Nothing
855646,855646,translate polish to english,Nie oddam statku.,I'm not surrendering this ship.
452580,452580,translate polish to english,Po prostu... Proszę.,Just... please.
683679,683679,translate english to polish,And it will be through the intervention of the...,Przez interwencje tych z poza drugiej strony z...
1625557,1625557,translate english to polish,He had one scene with Salman Khan in Dabangg 2...,"Zagrał w jednej scenie z Salmanem Khanem, a ma..."
182640,182640,translate polish to english,"Zabranie żołnierzy w ten region, gdy Haengin c...",You mustn't mobilize the troops when Haengin i...
391615,391615,translate english to polish,"KEY WORDS: China, trade exchange, import, expo...","SŁOWA KLUCZOWE: Chiny, wymiana handlowa, impor..."
1054548,1054548,translate polish to english,"Jesteśmy zaszczycenie, że mogliście przyjść.",We're so pleased you could come.
1885938,1885938,translate polish to english,"Jak już mówiłem reporterom, każdy w mieście wi...","Like I told those TV reporters, everybody in t..."
1681549,1681549,translate english to polish,Unhindered and safe access to victims is essen...,Możliwość bezpiecznego dotarcia bez utrudnień ...


In [ ]:
eval_df.sample(10)

,Unnamed: 0,prefix,input_text,target_text
8100,8100,translate polish to english,Wolę latać samolotem.,I prefer to travel by air.
8077,8077,translate english to polish,I'm losing my patience with you.,Tracę przy tobie cierpliwość.
1299,1299,translate english to polish,I leave for London tomorrow morning.,Jutro rano wyjeżdżam do Londynu.
16339,16339,translate english to polish,"One, three, and five are odd numbers.","Jeden, trzy i pięć to liczby nieparzyste."
6828,6828,translate polish to english,Wiatr pod wieczór uspokoił się.,The wind calmed down in the evening.
14594,14594,translate polish to english,"Nie ważne jak bardzo się staram, nie potrafię ...","No matter how hard I strive, I fail to achieve..."
8457,8457,translate english to polish,I also like cake.,Lubię też ciasto.
3330,3330,translate polish to english,Przestań kłamać.,Stop lying.
14246,14246,translate polish to english,"Nie mogłem ani jeść, ani pić.",I could neither eat nor drink.
16230,16230,translate polish to english,Gdzie jest dział dziecięcy?,Where's the children's section?


In case some extra column appears kill it

In [ ]:
def kill_col(df):
        if 'Unnamed: 0' in df:
            del df['Unnamed: 0']
#         print()
#         if 'index' in df:
#             df= df.drop(['index'], axis=1)

            
        return df

In [ ]:
eval_df= kill_col(eval_df)
train_df= kill_col(train_df)

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 3 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   prefix       object
 1   input_text   object
 2   target_text  object
dtypes: object(3)
memory usage: 45.8+ MB


In [ ]:
eval_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   prefix       20000 non-null  object
 1   input_text   20000 non-null  object
 2   target_text  20000 non-null  object
dtypes: object(3)
memory usage: 468.9+ KB


there were strange errors suggesting that column is not str type so I forced that above

In [ ]:
train_df.shape, eval_df.shape

((2000000, 3), (20000, 3))

## 2.Loading transformers library for mT5

https://github.com/google-research/multilingual-t5

In [ ]:
!pip install simpletransformers

In [ ]:
import logging
# import pandas as pd
from simpletransformers.t5 import T5Model, T5Args


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
model_args = T5Args()
model_args.max_seq_length = 96
model_args.train_batch_size = 10 
model_args.eval_batch_size = 10
model_args.num_train_epochs = 1
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 30000
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = "MT5 Polish-English Translation"

model = T5Model("mt5", "google/mt5-base", args=model_args)

## 3. Training model

In [ ]:
# Train the model
model.train_model(train_df, eval_data=eval_df)

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/2000000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


with batch size 20 trainig crashe so I cahnged that to 10 step 2

In [ ]:
# second attempt Train the model with batc size 10
model.train_model(train_df, eval_data=eval_df)

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/2000000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


https://simpletransformers.ai/docs/tips-and-tricks/#saving-checkpoints

Next attempts with othe rparameters will be done to train the model